In [1]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import sys
import os

print("Dependencies installed and imported.")

Dependencies installed and imported.


In [2]:

# Cell 2: Configuration, Data Loading, and Utility Functions

# --- Configuration for Falcon 7B Instruct ---
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

# --- Data Loading ---
try:
    # Adjust paths if your files are in a specific folder, otherwise assume root
    df1 = pd.read_csv("Health_Dataset_1_Sample.csv")
    df2 = pd.read_csv("Health_Dataset_2_Sample.csv")
    print("✅ Datasets loaded successfully.")
except FileNotFoundError as e:
    print(f"\n❌ Error: {e}. Please ensure the CSV files are uploaded/in the correct directory.")
    df1 = pd.DataFrame({'Patient_Number': ['101'], 'Age': [60], 'Gender': ['M']})
    df2 = pd.DataFrame({'Patient_Number': ['101'], 'Cholesterol': [200], 'Lifestyle_Score': [7]})
    # sys.exit(1) # Uncomment this if you want the script to stop completely on error.


# Ensure 'Patient_Number' is treated as a string for consistent merging/filtering
df1["Patient_Number"] = df1["Patient_Number"].astype(str)
df2["Patient_Number"] = df2["Patient_Number"].astype(str)

# --------------------

def get_merged_patient_record(patient_number: int):
    """Merges two dataframes and extracts a single patient's record."""
    merged = df1.merge(df2, on="Patient_Number", how="left")
    result = merged[merged["Patient_Number"] == str(patient_number)]

    if result.empty:
        return None

    # Return the first (and should be only) row as a dictionary
    return result.iloc[0].to_dict()

def build_prompt(patient_record, user_question):
    """
    Constructs the prompt using the LLM instruction template.
    (Note: Falcon-7B-Instruct uses a simple instruction format, unlike Mistral's <s>[INST]...[/INST])
    The template is slightly adjusted for Falcon's expected input structure for better results.
    """
    # Format the patient data into a readable list
    record_str = "\n".join([f"- {k.replace('_', ' ')}: {v}" for k, v in patient_record.items()])

    # Construct the instruction template
    prompt = f"""The following is patient health data. Analyze it and provide non-diagnostic, educational insights based ONLY on the provided data. DO NOT use diagnostic terms.

Patient Data:
{record_str}

User Question:
{user_question}

Assistant's Insight:
"""
    return prompt

print("Configuration and utility functions defined.")

✅ Datasets loaded successfully.
Configuration and utility functions defined.


In [3]:
# Cell 3: Model Initialization (Heavy Operation)

# --- Model Initialization ---
try:
    print(f"Loading model: {MODEL_NAME}...")

    # Load in 4-bit to save VRAM (requires bitsandbytes and accelerate)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto", # <-- Accelerate manages device placement
    )
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Use the 'text-generation' pipeline
    QA_PIPELINE = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )
    print("✅ Model loaded successfully.")

except Exception as e:
    print(f"\n❌ ERROR: Could not load the model '{MODEL_NAME}'.")
    print(f"Details: {e}")
    print("Please ensure you have sufficient VRAM/RAM (16GB+ recommended for 4-bit loading) and the necessary libraries.")
    sys.exit(1)

Loading model: microsoft/Phi-3-mini-4k-instruct...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


✅ Model loaded successfully.


In [4]:
# Cell 4: LLM Query Function

def ask_llm_for_explanation(prompt):
    """Generates the explanation using the text-generation pipeline."""

    response = QA_PIPELINE(
        prompt,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        pad_token_id=tokenizer.pad_token_id,
        # Stop generation once the model tries to generate another prompt/turn
        # Falcon models can sometimes repeat the prompt or continue the conversation
        # You may need to fine-tune this based on model behavior.
        stop_sequence=["User Question:", "Patient Data:"],
    )

    full_text = response[0]['generated_text']

    # Simple cleaning: remove the original prompt text from the full output
    # This assumes the prompt is at the start of the output
    if full_text.startswith(prompt):
         # Extract text after the prompt
        cleaned_text = full_text[len(prompt):].strip()
        # Optionally clean up any remnants of the 'stop_sequence' if it was generated
        return cleaned_text.split("User Question:")[0].split("Patient Data:")[0].strip()

    return full_text.strip()

print("LLM query function defined.")

LLM query function defined.


In [5]:
# Cell 5: Main Execution Block

print("\n" + "="*70)
print("--- Patient Health Analytics System (Falcon 7B Instruct) ---")
print("="*70)

# Use error handling for input
try:
    # Use raw input function for notebook
    patient_number_input = input("Enter Patient Number (e.g., 42, 101, 205): ")
    patient_number = int(patient_number_input)
except ValueError:
    print("❌ Invalid input for Patient Number. Please enter a number.")
    sys.exit(1)

user_question = input("Ask a question about the patient (e.g., 'What insights can be drawn from the patient's lifestyle factors?'): ")

# 1. Fetch the merged record
record = get_merged_patient_record(patient_number)

if record is None:
    print(f"❌ Patient number {patient_number} not found in the combined dataset.")
else:
    # 2. Build the detailed prompt for the LLM
    prompt = build_prompt(record, user_question)

    # 3. Get the explanatory answer from the LLM
    print("\n🧠 Generating AI Response... (This may take a moment)")
    answer_text = ask_llm_for_explanation(prompt)

    # 4. Print the result
    print("\n" + "="*70)
    print(f"🤖 AI Response (Falcon 7B) for Patient {patient_number}:")
    print("="*70)
    print(f"**Question:** {user_question}")
    print(f"**Model Insight:**\n{answer_text}")
    print("="*70)


--- Patient Health Analytics System (Falcon 7B Instruct) ---
Enter Patient Number (e.g., 42, 101, 205): 42
Ask a question about the patient (e.g., 'What insights can be drawn from the patient's lifestyle factors?'): can the patient have high chance of heart attack? Also give me some insights on his health.

🧠 Generating AI Response... (This may take a moment)

🤖 AI Response (Falcon 7B) for Patient 42:
**Question:** can the patient have high chance of heart attack? Also give me some insights on his health.
**Model Insight:**
This patient has a relatively normal BMI and a low level of stress, which are generally positive indicators for heart health. However, the salt intake in the diet is quite high, and daily alcohol consumption is also on the higher side. Both of these factors can contribute to high blood pressure and other heart health issues over time. It's important for the patient to manage their diet and limit alcohol intake as part of heart-healthy living.

In terms of hemoglobi

# Instruction Fine Tuning

In [6]:
def build_prompt(patient_record, user_question):
    """
    Constructs the prompt using the Mistral instruction template: <s>[INST]...[/INST]
    """
    # Format the patient data into a readable list
    # Use key:value format, replacing underscores for readability
    record_str = "\n".join([f"- {k.replace('_', ' ')}: {v}" for k, v in patient_record.items()])

    # Construct the Mistral instruction template
    prompt = f"""<s>[INST]
You are a health analytics assistant. Your goal is to provide non-diagnostic, educational insights based ONLY on the provided patient data.
DO NOT use terms like 'diagnosis,' 'disease,' or 'treatment.'

Patient Data:
{record_str}

User Question:
{user_question}

Instructions:
1. Identify relevant data points.
2. Produce medically safe, non-diagnostic insights.
3. Explain your reasoning clearly, linking the insight back to the data.
4. Keep the response concise and focused.
[/INST]"""
    return prompt


In [7]:
def build_prompt(patient_record, user_question):
    # Convert patient record to readable bullet points
    record_str = "\n".join([f"- {k.replace('_', ' ')}: {v}" for k, v in patient_record.items()])

    prompt = f"""
You are a medical data assistant. Answer the user's question using ONLY the patient data provided.
Be concise, factual, and avoid any medical diagnosis or treatment suggestions.

INSTRUCTION RULES:
1. Start with a **direct answer** to the user's question based solely on the patient data.
2. ONLY IF the user's question requires deeper context or if the patient data contains noteworthy values
   (unusually high, low, or potentially concerning), then add:
      Additional Insight (non-diagnostic): <brief clarification or risk awareness>
3. Do NOT:
   - invent information,
   - provide medical advice,
   - describe your reasoning steps,
   - speculate beyond the data.

REFERENCE:
- Genetic Pedigree Coefficient (GPC) ranges 0 to 1:
  • Closer to 0 → distant disease occurrence in family
  • Closer to 1 → immediate occurrence in family

PATIENT DATA:
{record_str}

USER QUESTION:
{user_question}

REQUIRED OUTPUT FORMAT:
Answer: <direct answer>
Additional Insight (non-diagnostic): <only if applicable>

Final Response:
""".strip()

    return prompt


In [8]:
def build_prompt(patient_record, user_question):
    record_str = "\n".join([f"- {k.replace('_', ' ')}: {v}" for k, v in patient_record.items()])

    prompt = f"""
You are a medical data assistant.

Your job is to answer the user's question using ONLY the patient data below.
Do NOT include instructions, rules, or explanations in your response.

Respond with:
Answer: <direct answer>
Additional Insight (non-diagnostic): <only if relevant and based on data>

Rules (the model must follow these silently):
- Base the answer strictly on the patient data.
- Do NOT repeat or mention these rules in the output.
- No medical diagnosis, no treatment advice.
- Only provide "Additional Insight" if the question needs deeper context OR if the data has clearly unusual values.
- Insights must be descriptive and NON-diagnostic (e.g., “higher than typical”, “could indicate”, “may relate to lifestyle factors”).
- Do NOT invent data, do NOT describe your reasoning.

Patient Data:
{record_str}

User Question: {user_question}

Now produce ONLY the formatted answer:
Answer:
Additional Insight (non-diagnostic):
""".strip()

    return prompt


In [11]:
def build_prompt(patient_record, user_question):
    record_str = "\n".join([f"- {k.replace('_', ' ')}: {v}" for k, v in patient_record.items()])

    prompt = f"""
You are a medical data assistant who answers clearly using ONLY the patient data.

Do not repeat instructions, do not state rules, and do not provide medical diagnosis or treatments.
If needed, you may add a brief non-diagnostic insight only when the data clearly shows unusual values or when the user's question requires context.

Patient Data:
{record_str}

User Question: {user_question}

Provide the response ONLY in this format:

Answer: <direct answer>
Additional Insight (non-diagnostic): <only if needed>
""".strip()

    return prompt


# Final Model

In [12]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import sys
import os

# --- Configuration for Mistral 7B Instruct ---
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
#MODEL_NAME = "tiiuae/falcon-7b-instruct"

# --- Dummy Data Setup (for a runnable script if files are missing) ---
# NOTE: Replace this block with your actual file loading if you have the CSVs.
try:
    df1 = pd.read_csv("Health_Dataset_1_Sample.csv")
    df2 = pd.read_csv("Health_Dataset_2_Sample.csv")
except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure the CSV files are in the correct directory.")
    # Exit or create dummy DataFrames for demonstration if needed
    # For a local fix, replace with your actual file paths or create dummy data.
    exit()
    df1 = pd.DataFrame(data1)
    df2 = pd.DataFrame(data2)

# Ensure 'Patient_Number' is treated as a string for consistent merging/filtering
df1["Patient_Number"] = df1["Patient_Number"].astype(str)
df2["Patient_Number"] = df2["Patient_Number"].astype(str)
# --------------------

def get_merged_patient_record(patient_number: int):
    """Merges two dataframes and extracts a single patient's record."""
    merged = df1.merge(df2, on="Patient_Number", how="left")
    result = merged[merged["Patient_Number"] == str(patient_number)]

    if result.empty:
        return None

    # Return the first (and should be only) row as a dictionary
    return result.iloc[0].to_dict()

# --- Model Initialization (FIXED) ---
try:
    print(f"Loading model: {MODEL_NAME}...")
    # 1. Load the Causal Language Model with device_map="auto"
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" # <-- Accelerate manages device placement
        # For lower memory usage (requires bitsandbytes/accelerate):
        # load_in_4bit=True
    )
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # 2. Use the 'text-generation' pipeline
    QA_PIPELINE = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        # FIX APPLIED: Removed the conflicting 'device' argument
    )
    print("Model loaded successfully.")

    def ask_llm_for_explanation(prompt):
        """Generates the explanation using the Mistral-tuned text-generation pipeline."""

        response = QA_PIPELINE(
            prompt,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            pad_token_id=tokenizer.pad_token_id
        )

        # Cleanly extract the generated text after the [/INST] tag
        full_text = response[0]['generated_text']
        print(f"This is the respoince of model{response[5:]}")
        if '[/INST]' in full_text:
            return full_text.split('[/INST]', 1)[-1].strip()

        return full_text.replace(prompt, '').strip()

except Exception as e:
    print(f"\n❌ ERROR: Could not load the Mistral model '{MODEL_NAME}'.")
    print(f"Details: {e}")
    print("Please ensure you have sufficient VRAM/RAM (8GB+ recommended) and the necessary libraries (transformers, torch, accelerate, bitsandbytes for 4-bit loading).")
    sys.exit(1)

# --- Prompt Building Function (Using Mistral's required format) ---
def build_prompt(patient_record, user_question):
    record_str = "\n".join([f"- {k.replace('_', ' ')}: {v}" for k, v in patient_record.items()])
    prompt = f"""
You are a medical data assistant. Answer the user's question directly using ONLY the patient data.

RULES:
- First, provide ONLY the direct answer to the user's question.
- If the data shows values that are unusually high, low, or risky, you may add an additional section:
  "Additional Insight (non-diagnostic): ..."
- Insights must NOT be diagnostic or prescriptive.
- Do NOT add chain-of-thought, follow-up questions, or long explanations.
- Do NOT continue the conversation beyond the answer and optional insight.
- Genetic Pedigree Coefficient (GPC) of an individual for a particular disease is a continuum between 0 and 1, where:
GPC closer to 0 indicates very distant occurrence of that disease in her/his pedigree, and
GPC closer to 1 indicates very immediate occurrence of that disease in her/his pedigree]


Patient Data:
{record_str}

User Question: {user_question}

FORMAT:
Answer: <the direct answer>
Additional Insight (non-diagnostic): <only if applicable>

Return nothing else.

Final Response:"""
    return prompt.strip()

# --- Main Execution Block ---

print("\n" + "="*70)
print("--- Patient Health Analytics System (Mistral 7B Instruct) ---")
print("="*70)

# Use error handling for input
try:
    patient_number_input = input("Enter Patient Number (e.g., 42, 101, 205): ")
    patient_number = int(patient_number_input)
except ValueError:
    print("❌ Invalid input for Patient Number. Please enter a number.")
    sys.exit(1)

user_question = input("Ask a question about the patient (e.g., 'What insights can be drawn from the patient's lifestyle factors?'): ")

# 1. Fetch the merged record
record = get_merged_patient_record(patient_number)

if record is None:
    print(f"❌ Patient number {patient_number} not found in the combined dataset.")
else:
    # 2. Build the detailed prompt for the LLM
    prompt = build_prompt(record, user_question)

    # 3. Get the explanatory answer from the LLM
    print("\n🧠 Generating AI Response... (This may take some time depending on your hardware)")
    answer_text = ask_llm_for_explanation(prompt)

    # 4. Print the result
    print("\n" + "="*70)
    print(f"🤖 AI Response (Mistral 7B) for Patient {patient_number}:")
    print("="*70)
    print(f"**Question:** {user_question}")
    print(f"**Model Insight:**\n{answer_text}")
    print("="*70)

Loading model: microsoft/Phi-3-mini-4k-instruct...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Model loaded successfully.

--- Patient Health Analytics System (Mistral 7B Instruct) ---
Enter Patient Number (e.g., 42, 101, 205): 41
Ask a question about the patient (e.g., 'What insights can be drawn from the patient's lifestyle factors?'): can the patient have high chance of heart attack? Also give me some insights on his health.

🧠 Generating AI Response... (This may take some time depending on your hardware)
This is the respoince of model[]

🤖 AI Response (Mistral 7B) for Patient 41:
**Question:** can the patient have high chance of heart attack? Also give me some insights on his health.
**Model Insight:**
Answer:
Additional Insight:
- The patient has a GPC of 0.87 for the disease in question, which indicates a high occurrence of that disease in their family history.
- The patient's age is 45, which puts them in a higher risk category for heart disease compared to younger individuals.
- The patient's salt content in the diet is relatively high at 2635, which can contribute to hi

In [14]:
!pip install -U ragas datasets evaluate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.9/419.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 124.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 M

# Evaluation

In [19]:
import pandas as pd
from datasets import Dataset
from typing import List, Dict, Any
import re

# A single test case for demonstration
eval_data = [
    {
        "question": "Can the patient have high chance of heart attack? Also give me some insights on his health.",
        "patient_record": { # The CONTEXT the LLM used (or should use)
            "Age": 55, "Gender": "M", "BMI": 31.5, "Blood_Pressure": "145/95",
            "Cholesterol": 230, "Smoking": "Yes", "GPC_Heart_Attack": 0.85
        },
        # The ACTUAL OUTPUT generated by your Phi-3 model
        "generated_answer": "Answer: The patient has a high Genetic Pedigree Coefficient (GPC) for Heart Attack (0.85), indicating an immediate family history risk. Additional Insight (non-diagnostic): The patient's BMI (31.5) and Blood Pressure (145/95) are in high ranges, and they report smoking, which are poor lifestyle factors. You should advise a doctor for a treatment plan.",
    },
    # Add 5-10 more test cases here for meaningful evaluation
]

# Convert the patient record dictionary into the context list format required by Ragas
def dict_to_context_list(record: Dict[str, Any]) -> List[str]:
    return ["\n".join([f"- {k.replace('_', ' ')}: {v}" for k, v in record.items()])]

# Create the final Ragas Dataset
df_eval = pd.DataFrame(eval_data)
df_eval['contexts'] = df_eval['patient_record'].apply(dict_to_context_list)
df_eval['answer'] = df_eval['generated_answer']

ragas_dataset = Dataset.from_pandas(df_eval[['question', 'contexts', 'answer']])

print(f"Evaluation dataset prepared with {len(ragas_dataset)} sample(s).")

Evaluation dataset prepared with 1 sample(s).


In [20]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy

# --- LLM-as-a-Judge Execution ---
# NOTE: Ragas requires an LLM to act as the judge. For this to run immediately,
# you typically need an OpenAI/Anthropic API key set as environment variables.
# Without an API key, Ragas may default to an open-source model, but that requires
# more complex setup. Assuming API keys are set for simplicity.

ragas_results = evaluate(
    ragas_dataset,
    metrics=[
        faithfulness,      # Checks Groundedness: Is the answer supported by the context? (Crucial for Factual Accuracy)
        answer_relevancy   # Checks Relevance: Does the answer address the question?
    ]
)

print("\n--- RAGAS Scores (Higher is Better) ---")
print(ragas_results)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

# For Evaluation I don't have api key for ragas which check for guidrels, consistency , releavnece